In [1]:
from transformers import pipeline
import pandas as pd

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('./review_data/뱅뱅막국수_reviews_추천순.csv')

In [3]:
data

,Date,Review
0,2024-11,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...
1,2024-11,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛...
2,2024-11,처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고\n양...
3,2024-10,뱅뱅막국수!\n강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 ...
4,2024-10,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 🍜✨\n\n오늘 간만에 와이프와 늦은 점심...
...,...,...
983,2024-08,참기름 막국수랑 육전 조합 최고에요!!\n맛집으로 추천합니다❤️
984,2024-04,저녁시간에 가서 웨이팅 있었습니다. 자극적이지 않고 맛있었어요 :)
985,2024-07,깔끔하고 단백하고 너무 맛있어요 ^^
986,2024-07,양재역 근처 맛집입니다!! 우연히 들어가서 먹었는데 너무 맛있었어요 ㅎㅎ 더운 여름...


In [4]:
sentiment_analyzer = pipeline("sentiment-analysis", model="beomi/kcbert-base")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
data['sentiment'] = data['Review'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

In [7]:
data[['Review', 'sentiment']]

,Review,sentiment
0,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...,LABEL_1
1,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛...,LABEL_1
2,처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고\n양...,LABEL_1
3,뱅뱅막국수!\n강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 ...,LABEL_1
4,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 🍜✨\n\n오늘 간만에 와이프와 늦은 점심...,LABEL_1
...,...,...
983,참기름 막국수랑 육전 조합 최고에요!!\n맛집으로 추천합니다❤️,LABEL_1
984,저녁시간에 가서 웨이팅 있었습니다. 자극적이지 않고 맛있었어요 :),LABEL_0
985,깔끔하고 단백하고 너무 맛있어요 ^^,LABEL_1
986,양재역 근처 맛집입니다!! 우연히 들어가서 먹었는데 너무 맛있었어요 ㅎㅎ 더운 여름...,LABEL_1


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import re

# 데이터 전처리 함수 정의
def preprocess_text(text):
    # 사용자 선호 전처리 방식: 영어, 숫자, 한국어, 줄바꿈 문자만 남기고 나머지 제거
    text = re.sub(r'[^a-zA-Z0-9가-힣\s\n]', '', text)
    text = re.sub(r'\s+', ' ', text)  # 중복 공백 제거
    return text.strip()

# 감정 분석에 사용할 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)  # 긍정/부정 두 가지 레이블 사용

# 감정 분석 파이프라인 생성
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 리뷰 데이터를 로드하고 전처리 (리뷰 파일 경로에 맞게 파일명을 변경)
data = pd.read_csv('./review_data/뱅뱅막국수_reviews_추천순.csv')
data['cleaned_review'] = data['Review'].apply(preprocess_text)

# 감정 분석 수행
data['sentiment'] = data['cleaned_review'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

# 결과 확인
data[['Review', 'cleaned_review', 'sentiment']]


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Review,cleaned_review,sentiment
0,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...,LABEL_0
1,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛...,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요진짜 점심때 사람이 많아 맛집...,LABEL_1
2,처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고\n양...,처음 방문해봤는데 웨이팅이 있을만하네요 애매한 시간에 가서 바로 들어갔고 양도 진짜...,LABEL_0
3,뱅뱅막국수!\n강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 ...,뱅뱅막국수 강남 도곡동 뱅뱅사거리 막국수 맛집이네요뱅뱅 돌린 메밀면에 돌돌 말은 예...,LABEL_1
4,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 🍜✨\n\n오늘 간만에 와이프와 늦은 점심...,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 오늘 간만에 와이프와 늦은 점심과 낮술이 ...,LABEL_1
...,...,...,...
983,참기름 막국수랑 육전 조합 최고에요!!\n맛집으로 추천합니다❤️,참기름 막국수랑 육전 조합 최고에요 맛집으로 추천합니다,LABEL_1
984,저녁시간에 가서 웨이팅 있었습니다. 자극적이지 않고 맛있었어요 :),저녁시간에 가서 웨이팅 있었습니다 자극적이지 않고 맛있었어요,LABEL_0
985,깔끔하고 단백하고 너무 맛있어요 ^^,깔끔하고 단백하고 너무 맛있어요,LABEL_0
986,양재역 근처 맛집입니다!! 우연히 들어가서 먹었는데 너무 맛있었어요 ㅎㅎ 더운 여름...,양재역 근처 맛집입니다 우연히 들어가서 먹었는데 너무 맛있었어요 더운 여름에 시원하...,LABEL_1


In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import re

# 데이터 전처리 함수 정의
def preprocess_text(text):
    # 사용자 선호 전처리 방식: 영어, 숫자, 한국어, 줄바꿈 문자만 남기고 나머지 제거
    text = re.sub(r'[^a-zA-Z0-9가-힣\s\n]', '', text)
    text = re.sub(r'\s+', ' ', text)  # 중복 공백 제거
    return text.strip()

# 감정 분석에 사용할 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModelForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=2)  # 긍정/부정 두 가지 레이블 사용

# 감정 분석 파이프라인 생성
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# 리뷰 데이터를 로드하고 전처리 (리뷰 파일 경로에 맞게 파일명을 변경)
data = pd.read_csv('./review_data/뱅뱅막국수_reviews_추천순.csv')  # 리뷰 데이터 파일 경로에 맞게 파일명을 변경
data['cleaned_review'] = data['Review'].apply(preprocess_text)

# 감정 분석 수행
data['sentiment'] = data['cleaned_review'].apply(lambda x: sentiment_analyzer(x)[0]['label'])

# 결과 확인
print(data[['Review', 'cleaned_review', 'sentiment']])


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


                                                Review  \
0    그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...   
1    도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛...   
2    처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고\n양...   
3    뱅뱅막국수!\n강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 ...   
4    강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 🍜✨\n\n오늘 간만에 와이프와 늦은 점심...   
..                                                 ...   
983                참기름 막국수랑 육전 조합 최고에요!!\n맛집으로 추천합니다❤️   
984              저녁시간에 가서 웨이팅 있었습니다. 자극적이지 않고 맛있었어요 :)   
985                               깔끔하고 단백하고 너무 맛있어요 ^^   
986  양재역 근처 맛집입니다!! 우연히 들어가서 먹었는데 너무 맛있었어요 ㅎㅎ 더운 여름...   
987                                  너무너무 맛있어요 4번째 왔어요   

                                        cleaned_review sentiment  
0    그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...   LABEL_0  
1    도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요진짜 점심때 사람이 많아 맛집...   LABEL_0  
2    처음 방문해봤는데 웨이팅이 있을만하네요 애매한 시간에 가서 바로 들어갔고 양도 진짜...   LABEL_0  
3    뱅뱅막국수 강남 도곡동 뱅뱅사거리 막국수 맛집이네요뱅뱅

In [13]:
data[data['sentiment']=='LABEL_0']

,Date,Review,cleaned_review,sentiment
0,2024-11,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...,그 유명한 맛집 뱅뱅막국수 다녀왔습니당 주문하고 바로 뽑아내는 메밀 자가제면이라 면...,LABEL_0
1,2024-11,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요.진짜 점심때 사람이 많아 맛...,도곡동맛집 뱅뱅막국수는 유튜브 먹방에서 보고 찾아갔어요진짜 점심때 사람이 많아 맛집...,LABEL_0
2,2024-11,처음 방문해봤는데.. 웨이팅이 있을만하네요👍 애매한 시간에 가서 바로 들어갔고\n양...,처음 방문해봤는데 웨이팅이 있을만하네요 애매한 시간에 가서 바로 들어갔고 양도 진짜...,LABEL_0
3,2024-10,뱅뱅막국수!\n강남 도곡동 뱅뱅사거리 막국수 맛집!이네요.뱅뱅 돌린 메밀면에 돌돌 ...,뱅뱅막국수 강남 도곡동 뱅뱅사거리 막국수 맛집이네요뱅뱅 돌린 메밀면에 돌돌 말은 예...,LABEL_0
4,2024-10,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 🍜✨\n\n오늘 간만에 와이프와 늦은 점심...,강남역 최애 맛집 뱅뱅막국수 다녀왔습니다 오늘 간만에 와이프와 늦은 점심과 낮술이 ...,LABEL_0
...,...,...,...,...
983,2024-08,참기름 막국수랑 육전 조합 최고에요!!\n맛집으로 추천합니다❤️,참기름 막국수랑 육전 조합 최고에요 맛집으로 추천합니다,LABEL_0
984,2024-04,저녁시간에 가서 웨이팅 있었습니다. 자극적이지 않고 맛있었어요 :),저녁시간에 가서 웨이팅 있었습니다 자극적이지 않고 맛있었어요,LABEL_0
985,2024-07,깔끔하고 단백하고 너무 맛있어요 ^^,깔끔하고 단백하고 너무 맛있어요,LABEL_0
986,2024-07,양재역 근처 맛집입니다!! 우연히 들어가서 먹었는데 너무 맛있었어요 ㅎㅎ 더운 여름...,양재역 근처 맛집입니다 우연히 들어가서 먹었는데 너무 맛있었어요 더운 여름에 시원하...,LABEL_0
